In [ ]:
%autosave 0

In [ ]:
from __future__ import print_function, absolute_import, division
!umask 2
import logging
import os
import sys
from collections import defaultdict, OrderedDict
from errno import ENOENT, ENODATA
from stat import S_IFDIR, S_IFLNK, S_IFREG
from sys import argv, exit, path
from time import time, sleep
from datetime import datetime
import struct as pystruct
from binascii import hexlify

sys.path.insert(0,"../tagfuse")
from tagfuse import radioutils
from tagfuse import TagFuseTree
from tagfuse import taghandlers

sys.path.insert(0,"../tagnet")
from tagnet import *

from fuse import FuseOSError

if not hasattr(__builtins__, 'bytes'):
    bytes = str

In [ ]:
import inspect

In [ ]:
from tagfuse import tagfuseargs
sys.argv = ['tagfuse.py', '~/tags', '--disable_sparse']
tagfuseargs.process_cmd_args()
tagfuseargs.get_cmd_args()

In [ ]:
from TagFuseTree import TagFuseRootTree, TagFuseTagTree

In [ ]:
#tagfuse.tagfuseargs.global_args=None

In [ ]:
#radio = radio_start()
from si446x import Si446xRadio
from si446x import get_config_wds, get_name_wds, wds_default_config
radio=Si446xRadio(0)
if (radio == None):
    raise RuntimeError('radio_start: could not instantiate radio')
radio.unshutdown()
wds_default_config(0) # force alternate default config
radio.write_config()
radio.config_frr()
radio.set_property('PKT', 0x0b, '\x28\x28') # tx/rx threshold
radio.set_property('PREAMBLE', 0, '\x40')   # long preamble

## FUNCTION AND VARIABLE DEFINITIONS

In [ ]:
tree_root = TagFuseRootTree(radio)

In [ ]:
def LocateNode(tag_tree, path):
    # print(path2list(path))
    if (path == '/'):
        print('located root')
        return tree_root
    return tag_tree.traverse(radioutils.path2list(path), 0)

def DeleteNode(path, node):
    pass

### Image Info Metadata definitions from TAG code
```
typedef struct {                        /* little endian order  */
  uint16_t build;                       /* that's native for us */
  uint8_t  minor;
  uint8_t  major;
} image_ver_t;

typedef struct {
  uint8_t  hw_rev;
  uint8_t  hw_model;
} hw_ver_t;

typedef struct {
  uint32_t    ii_sig;                   /*  b  must be IMAGE_INFO_SIG to be valid */
  uint32_t    image_start;              /*  b  where this binary loads            */
  uint32_t    image_length;             /*  b  byte length of entire image        */
  image_ver_t ver_id;                   /*  b  version string of this build       */
  uint32_t    image_chk;                /*  s  simple checksum over entire image  */
  hw_ver_t    hw_ver;                   /*  b  2 byte hw_ver                      */
  uint8_t     reserved[10];             /*  b  reserved                           */
} image_info_basic_t;

typedef struct {
  uint16_t    tlv_block_len;
  uint8_t     tlv_block[IMG_INFO_PLUS_SIZE];
} image_info_plus_t;
```

In [ ]:
#  IMAGE_INFO provides information about a Tag software image. This data is
#  embedded in the image itself. The IMAGE_META_OFFSET is the offset into
#  the image where image_info lives in the image.  It directly follows the
#  exception vectors which are 0x140 bytes long.
# 
#  This struct will have to change, If MSP432 vector table length changes.
# 
IMAGE_INFO_SIG = 0x33275401
IMAGE_META_OFFSET = 0x140
IMAGE_INFO_DEFAULT = [IMAGE_INFO_SIG, 0x20000, (0x140*2)+0x1c, 
                      9999, 2, 0, 
                      0, 
                      99, 77, 
                      '\00'  * 10]
IMAGE_INFO_LEN = 2
#
# Struct created for accessing image info (little indian)
# sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major, main_tree, aux_tree, build_time, im_rev, im_model = image_info
#
IMB_FIELDS = '<LLLHBBLBB10s'
image_info_struct = pystruct.Struct(IMB_FIELDS)
IMAGE_MIN_SIZE  =  (IMAGE_META_OFFSET + image_info_struct.size)

In [ ]:
def info_check(filename):
    infile = open(filename, 'rb')
    if not infile:
        return (NONE, NONE, NONE)
    infile.seek(0, 2) # seek to the end
    file_size = infile.tell()
    if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    infile.seek(0, 0)    # seek to the beginnnig
    # get image info from input file and sanity check
    infile.seek(IMAGE_META_OFFSET) # seek to location of image info
    image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
    print("file information")
    sig, image_start, imagelength, im_build, im_minor, im_major, image_chk, \
         im_rev, im_model, pad = image_info
    pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, image_chk: 0x{:x}"
    print(pstr.format(sig, image_start, imagelength, image_chk))
    pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
    print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
    if sig != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    infile.seek(0)    # seek to the beginning
    return (infile, (im_major, im_minor, im_build), imagelength)

In [ ]:
def _file_size(fd):
    fd.seek(0, 2) # seek to the end
    _size = fd.tell()
    if _size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    fd.seek(0, 0)    # seek to the beginnnig
    return _size

In [ ]:
def default_image(filename):
    # write out simple default binary input file for testing purposes
    #
    if not os.path.isfile(filename):
        with open(filename,'wb') as outfile:
            buf = bytearray(IMAGE_META_OFFSET)
            for x in range(1,IMAGE_META_OFFSET): buf[x] = x & 0x0f
            outfile.write(buf)
            outfile.write(bytearray(image_info_struct.pack(*IMAGE_INFO_DEFAULT)))
            for i in range(8):
                for x in range(256): buf[x] = x & 0x7f
                outfile.write(buf)
            im_info = IMAGE_INFO_DEFAULT
            im_info[IMAGE_INFO_LEN] = outfile.tell()
            outfile.seek(IMAGE_META_OFFSET)
            outfile.write(bytearray(image_info_struct.pack(*IMAGE_INFO_DEFAULT)))
    else:
        print('file already exists', filename)

In [ ]:
def get_name_from_class(model):
    c = model.__class__.__mro__[0]
    name = c.__module__ + "." + c.__name__
    return name

## Write Image using TagFuse API

In [ ]:
def write_file(fd, handler, path_list, fsize):
    while (fsize - fd.tell() > 0):
        fpos = fd.tell()
        size = handler.write(path_list, fd.read(4096), fpos)

In [ ]:
def _file_size(fd):
    fd.seek(0, 2) # seek to the end
    _size = fd.tell()
    if _size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    fd.seek(0, 0)    # seek to the beginnnig
    return _size

##  Write Image using radioutils

In [ ]:
# default paramters
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
SHORT_DELAY         = 0

In [ ]:
class RadioLoadException(Exception):
    pass

In [ ]:
def upload_image(radio, tagpath, filename):
    
    def _file_size(fd):
        fd.seek(0, 2) # seek to the end
        _size = fd.tell()
        if _size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
        fd.seek(0, 0)    # seek to the beginnnig
        return _size

    def _put_msg(fd, path, version, file_size, eod=False):
        tlv_list = radioutils.path2tlvs(path_list)
        tlv_list.append(TagTlv(tlv_types.VERSION, version))
        if (fd.tell() > 0):
            tlv_list.append(TagTlv(tlv_types.OFFSET, fd.tell()))
        # build the PUT mesage object
        req_obj = TagPut(TagName(tlv_list))
        # optionally add payload
        if eod:
            # send end of data indication
            pload = TagTlvList([TagTlv(tlv_types.EOF)])
        elif (fd.tell() < file_size):
            # determine payload size to send and read it
            chunk_size = MAX_PAYLOAD - req_obj.pkt_len()
            if ((file_size - fd.tell()) < chunk_size):
                chunk_size = file_size - fd.tell()
            pload = bytearray(fd.read(chunk_size))
        # else
            # just send without payload
        if (pload is not None):
            req_obj.payload = pload
        return req_obj, len(pload)

    start = datetime.now()
    print(start)
    try:
        # open input file and determine its length
        print(filename)
        infile = open(filename, 'rb')
        file_size = _file_size(infile)

        # get image info from input file and sanity check
        infile.seek(IMAGE_META_OFFSET) # seek to location of image info
        image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
        print("file information")
        sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
            main_tree, aux_tree, build_time, im_rev, im_model = image_info
        pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
        print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
        pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
        print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
        if sig != IMAGE_INFO_SIG:
            raise RadioLoadException("image info is corrupted")
        if imagelength != file_size:
            raise RadioLoadException("file size doesn't match image info, file: {}, info: {}".format(
                                                                                        file_size,
                                                                                        imageLength))
        infile.seek(0)    # seek back to the beginnnig

        # loop to transfer image data to tag
        retries         = 0
        total_retries   = 0
        packets_sent    = 0
        while (file_size - infile.tell() > 0):
            if ((packets_sent % 10) == 0):
                print("\r{0:.2f}%".format((float(infile.tell())/float(file_size))*100), end='')
            #    print(">>>> file size: {}, offset: {}, retries: {}, rssi: {}\r".format(file_size, infile.tell(), retries, rssi, end=''))
            req_msg, plen = _put_msg(infile, tagpath, (im_major, im_minor, im_build), file_size)
            err, payload = radioutils.msg_exchange(radio, req_msg)
            packets_sent = packets_sent + 1
            if (err) and (err == tlv_errors.EALREADY):
                raise RadioLoadException('file already exists')
            offset = None
            if (err is None) or (err == tlv_errors.SUCCESS):
                offset = radioutils.payload2values(payload,
                                     [tlv_types.OFFSET,
                                     ])[0]
                # zzz print(offset, err)
            if offset and offset != infile.tell():
                if (err): p = "{}".format(err)
                else:     p = "BAD OFFSET"
                print("\n{}  {}! offset: response {}, expected {}".format(
                                                                        datetime.now(),
                                                                        p,
                                                                        offset,
                                                                        infile.tell()))
                if (offset >= 0):
                    infile.seek(offset, 0)
                else:
                    infile.seek(-plen, 1)
                    offset -= plen
                retries = retries + 1
                total_retries = total_retries + 1
            else:
                retries = 0
            if (retries > MAX_RETRIES):
                raise RadioLoadException("too many retries")
            sleep(SHORT_DELAY)

        # send end of file to complete the image load operation
        
        req_msg, plen = _put_msg(infile, tagpath, (im_major, im_minor, im_build), file_size, eod=True)
        err, payload = radioutils.msg_exchange(radio, req_msg)
        if (err is None) or (err == tlv_errors.SUCCESS):
            offset = radioutils.payload2values(payload,
                                     [tlv_types.OFFSET,
                                     ])[0]
#            if (offset is none): offset = -1
#        else:
#            offset = -1
        print("\ntotals bytes: {}, packets: {} retries: {}".format(offset,
                                                                   packets_sent,
                                                                   total_retries))
    except RadioLoadException as e:
        print('terminating:', e)
    finally:
        infile.close()
    print('\ndone, elapsed time', datetime.now() - start)

In [ ]:
STOP

In [ ]:
STOP

## Get Version of 'active', 'standby', 'golden', 'nib', and 'running' Images

In [ ]:
STOP

In [ ]:
def get_version(radio, path, which):
    
    def _get_version_msg(path, which):
        tlv_list = path2tlvs(path2list(path))
        tlv_list.append(TagTlv(tlv_types.STRING, which))
        req_obj = TagGet(TagName(tlv_list))
        return req_obj

    req_msg = _get_version_msg(path, which)
    # zzz print(req_msg.name)
    err, payload = msg_exchange(radio, req_msg)
    # zzz print(payload)
    if (err == None):
        err = tlv_errors.SUCCESS
    if err == tlv_errors.SUCCESS:
        version, state = payload2values(payload,
                                [tlv_types.VERSION,
                                 tlv_types.STRING,
                                ])
    else:
        version = (0,0,0)
        state = 'x'

    return err, version, state

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

In [ ]:
print(get_version(radio, TAG_PATH, 'active'))
print(get_version(radio, TAG_PATH, 'backup'))
print(get_version(radio, TAG_PATH, 'running'))
print(get_version(radio, TAG_PATH, 'nib',))
print(get_version(radio, TAG_PATH, 'golden'))

## Delete Version Image

In [ ]:
STOP

In [ ]:
def delete_image(radio, path, version):
    
    def _delete_msg(path, version):
        tlv_list = path2tlvs(path2list(path))
        tlv_list.append(TagTlv(tlv_types.VERSION, version))
        req_obj = TagDelete(TagName(tlv_list))
        return req_obj

    req_msg = _delete_msg(path, version)
    print(req_msg.name)
    err, payload = msg_exchange(radio, req_msg)
    print(payload)
    if (err is None):
        err = tlv_errors.SUCCESS
    return err

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sd/0/img'

In [ ]:
print(delete_image(radio, TAG_PATH, (0,2,386)))

## Set Version of 'active' and 'backup' Images

In [ ]:
STOP

In [ ]:
def set_version(radio, path, which, version):
    
    def _set_version_msg(path, which, version):
        tlv_list = path2tlvs(path2list(path))
        tlv_list.extend([
            TagTlv(tlv_types.STRING, which),
            TagTlv(tlv_types.VERSION, version),
           ])
        req_obj = TagPut(TagName(tlv_list))
        return req_obj

    req_msg = _set_version_msg(path, which, version)
    err, payload = msg_exchange(radio, req_msg)
    if (err is None):
        err = tlv_errors.SUCCESS
    return err

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

In [ ]:
print(set_version(radio, TAG_PATH, 'active', (0, 2, 384)))
#print(set_version(radio, TAG_PATH, 'backup', (0, 2, 386)))

## Get Poll Count and Events

In [ ]:
STOP

In [ ]:
def get_poll_info(radio, path, name):
    
    def _get_poll_msg(path, name):
        tlv_list = radioutils.path2tlvs(path2list(path))
        tlv_list.append(TagTlv(name))
        req_obj = TagHead(TagName(tlv_list))
        return req_obj

    req_msg = _get_poll_msg(path, name)
    err, payload = radioutils.msg_exchange(radio, req_msg)
    if (err is None) or (err == tlv_errors.SUCCESS):
        size = radioutils.payload2values(payload,
                                [tlv_types.SIZE,
                                ])[0]

        return size
    return None

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/poll'

In [ ]:
print('poll count', get_poll_info(radio, TAG_PATH, 'cnt'))
print('poll events', get_poll_info(radio, TAG_PATH, 'ev'))

## Reboot into 'active', 'standby', 'golden', 'nib', and 'running' Images

In [ ]:
STOP

In [ ]:
def reboot_version(radio, path, which, version):
    
    def _reboot_version_msg(path, which, version):
        tlv_list = radioutils.path2tlvs(radioutils.path2list(path))
        tlv_list.extend([
            TagTlv(tlv_types.STRING, which),
            TagTlv(tlv_types.VERSION, version),
           ])
        req_obj = TagPut(TagName(tlv_list))
        return req_obj

    req_msg = _reboot_version_msg(path, which, version)
    err, payload = radioutils.msg_exchange(radio, req_msg)
    if (err is None):
        err = tlv_errors.SUCCESS
    return err

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

In [ ]:
reboot_version(radio, TAG_PATH, 'active', (0, 1, 0))
#set_version(radio, TAG_PATH, 'backup', (0, 1, 0))
#set_version(radio, TAG_PATH, 'running', (0, 1, 0))
#set_version(radio, TAG_PATH, 'nib', (0, 1, 0))
reboot_version(radio, TAG_PATH, 'golden', (0, 1, 0))

## Examine Nodes in Fuse Tree

In [ ]:
STOP

In [ ]:
path='/'
handler = LocateNode(tag_tree, path)
print(type(handler))

In [ ]:
h = tree_root
isinstance(h, taghandlers.DirHandler)

In [ ]:
res = inspect.getmembers(h)
for x, y in res:
    if (x == '__class__'):
        break
print(y,type(y),type(h),type(taghandlers.DirHandler))
print(isinstance(h,y), isinstance(h,taghandlers.DirHandler), isinstance(y,taghandlers.DirHandler))
print(issubclass(type(h),y), issubclass(type(h),taghandlers.DirHandler), issubclass(type(y),taghandlers.DirHandler))

In [ ]:
h['']

In [ ]:
handler['']

In [ ]:
handlerA = LocateNode(tree_root ,'/ffffffffffff/tag/sd/0/dblk')
handlerA

In [ ]:
handlerB = tree_root['ffffffffffff']['tag']['sd']['0']['dblk']
handlerB

In [ ]:
print(handlerA, handlerB)
handlerA == handlerB

In [ ]:
tree_root['ffffffffffff']['tag']['sd']['0']['dblk']['byte']

In [ ]:
if isinstance(handlerA, taghandlers.DirHandler): print('ok')

In [ ]:
path='/ffffffffffff'
handler, path_list = LocateNode(tag_tree, path)
type(handler), path_list

In [ ]:
handler.getattr(path2list(path), update=True)

In [ ]:
path='/ffffffffffff/tag/sd'
handler, path_list = LocateNode(tag_tree, path)
type(handler), path_list

## MAIN BLOCK - START HERE

In [ ]:
STOP

## Examine Image Info

In [ ]:
!ls ../../tagbin

In [ ]:
BIN_ROOT    = '../../tagbin'
#FILENAME    = '/tmp/test.bin'
fname = '0.2.9999'
image_file_name    = BIN_ROOT + '/' + fname

In [ ]:
info_check(image_file_name)

In [ ]:
i_fd, i_vers, i_size = info_check(image_file_name)
#i_fd.close()
print('{}.{}.{}'.format(*i_vers),i_size, hex(i_size))

## Poll for Tags

In [ ]:
tagpath='/.poll'

In [ ]:
handler, path_list = LocateNode(tree_root, tagpath)
type(handler), path_list

In [ ]:
found_tags = handler.readdir(path_list, tree_root, TagFuseTagTree)

In [ ]:
tag_id = '658bc8e5205c'

## Examine Image Directory

In [ ]:
dirpath='/{}/tag/sd/0/img'.format(tag_id)
dirpath

In [ ]:
dirhandler, path_list = LocateNode(tree_root, dirpath)
type(dirhandler), path_list

In [ ]:
img_list = dirhandler.readdir(path_list, tree_root, TagFuseTagTree)
img_list

## Load image file using Fuse Handler

In [ ]:
version='0.2.9999'
filepath = '{}/{}'.format(dirpath, version)
file_list = radioutils.path2list(filepath)
dirpath, filepath, file_list

In [ ]:
mode = 0o0666
dirhandler.create(file_list, mode)

In [ ]:
dirhandler

In [ ]:
filehandler, file_list = LocateNode(tree_root, filepath)
type(filehandler), file_list

In [ ]:
i_fd.tell(), i_size

In [ ]:
i_fd.seek(0)

In [ ]:
write_file(i_fd, filehandler, file_list, i_size)

In [ ]:
file_list

In [ ]:
filehandler.flush(file_list)

In [ ]:
attrs = filehandler.getattr(path_list, update=True)
attrs

## Low level IO for testing Image Load

In [ ]:
STOP

In [ ]:
MAX_FIFO_SIZE       = 129
MAX_WAIT            = 1000 # milliseconds
MAX_RECV            = 2
MAX_PAYLOAD         = 254
MAX_RETRIES         = 4
RADIO_POWER         = 20
SHORT_DELAY         = 1000 # milliseonds

In [ ]:
def _put_msg(fd, path, version, file_size, eod=False):
    tlv_list = radioutils.path2tlvs(path_list)
    tlv_list.append(TagTlv(tlv_types.VERSION, version))
    if (fd.tell() > 0):
        tlv_list.append(TagTlv(tlv_types.OFFSET, fd.tell()))
    # build the PUT mesage object
    req_obj = TagPut(TagName(tlv_list))
    # optionally add payload
    if eod:
        # send end of data indication
        pload = TagTlvList([TagTlv(tlv_types.EOF)])
    elif (fd.tell() < file_size):
        # determine payload size to send and read it
        chunk_size = MAX_PAYLOAD - req_obj.pkt_len()
        if ((file_size - fd.tell()) < chunk_size):
            chunk_size = file_size - fd.tell()
        pload = bytearray(fd.read(chunk_size))
    # else
        # just send without payload
    if (pload is not None):
        req_obj.payload = pload
    return req_obj, len(pload)

In [ ]:
req_msg, plen = _put_msg(i_fd, filepath, i_vers, i_size)

In [ ]:
req_msg.name, plen, len(req_msg.payload)

In [ ]:
start = time()
sstatus = radioutils.radio_send_msg(radio, req_msg.build(), RADIO_POWER)
rsp_msg, rssi, rstatus = radioutils.radio_receive_msg(radio, MAX_RECV, MAX_WAIT*10)
time() - start, TagMessage(rsp_msg).payload if rsp_msg else None, rssi

In [ ]:
i_fd.tell(), hex(i_fd.tell())

In [ ]:
i_fd.seek(0)
start = time()
req_msg, plen = _put_msg(i_fd, filepath, i_vers, i_size)
sstatus = radioutils.radio_send_msg(radio, req_msg.build(), RADIO_POWER)
rsp_msg, rssi, rstatus = radioutils.radio_receive_msg(radio, MAX_RECV, MAX_WAIT*10)
time() - start, TagMessage(rsp_msg).payload if rsp_msg else None, rssi

In [ ]:
i_fd.tell(), hex(i_fd.tell())

In [ ]:
start = time()
req_msg, plen = _put_msg(i_fd, filepath, i_vers, i_size)
sstatus = radioutils.radio_send_msg(radio, req_msg.build(), RADIO_POWER)
rsp_msg, rssi, rstatus = radioutils.radio_receive_msg(radio, MAX_RECV, MAX_WAIT*10)
time() - start, TagMessage(rsp_msg).payload if rsp_msg else None, rssi

In [ ]:
i_fd.tell(), hex(i_fd.tell())

In [ ]:
start = time()
req_msg, plen = _put_msg(i_fd, filepath, i_vers, i_size, eod=True)
sstatus = radioutils.radio_send_msg(radio, req_msg.build(), RADIO_POWER)
rsp_msg, rssi, rstatus = radioutils.radio_receive_msg(radio, MAX_RECV, MAX_WAIT*10)
time() - start, TagMessage(rsp_msg).payload if rsp_msg else None, rssi

In [ ]:
i_fd.seek(1000)

## Create Default Image File

In [ ]:
STOP

In [ ]:
fname = '../../tagbin/0.2.9999'
if os.path.isfile(fname):
    print('deleting', fname)
    os.remove(fname)

In [ ]:
default_image(fname)

In [ ]:
!hexdump $fname

In [ ]:
fd, vers, length = info_check(fname)
fd, vers, length

In [ ]:
_file_size(fd)

In [ ]:
fd.close()